In [1]:
#PDF RAG QnA System

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
from langchain_community.document_loaders import PyPDFLoader

file_path = 'Analytical Science Advances - 2023 - Rodriguez - Recent advances of Raman spectroscopy for the analysis of bacteria.pdf'
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))

15


In [4]:
print(docs[0].page_content[0:200])
print(docs[0].metadata)

81 Analytical Science AdvancesMini Review
doi.org/10.1002/ansa.202200066
Received: 26 December 2022
Revised: 3 March 2023
Accepted: 10 March 2023
Recent advances of Raman spectroscopy for the analysis
{'source': 'Analytical Science Advances - 2023 - Rodriguez - Recent advances of Raman spectroscopy for the analysis of bacteria.pdf', 'page': 0}


In [5]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')

In [6]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=5000, chunk_overlap= 1000)
splits = text_splitter.split_documents(docs)
embed = OpenAIEmbeddings(show_progress_bar= True, request_timeout= 10000 )
vectorstore = Chroma.from_documents(documents= splits, embedding= embed, persist_directory= './c_db')

retriever = vectorstore.as_retriever()

  0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate(
    [
        ("system", system_prompt),
        ("human", "{input}")
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [11]:
results = rag_chain.invoke({"input": "Eplain how Raman spectroscopy is used for the analysis of microorganisms "})
results['answer']

  0%|          | 0/1 [00:00<?, ?it/s]

"Raman spectroscopy is used for the analysis of microorganisms by measuring the inelastic scattering of monochromatic light from a laser source as it interacts with the vibrational modes of molecules within the sample. This interaction produces a Raman shift, which provides a spectral fingerprint unique to the biochemical constituents of bacterial cells. Raman spectroscopy's non-destructive nature, combined with its ability to offer rapid, sensitive, and label-free detection, makes it a powerful tool for identifying and discriminating between different bacterial species and strains."